In [5]:
import sympy as sym
import numpy as np
from sympy.physics.quantum.tensorproduct import TensorProduct

# CHSH

$$
\begin{align}
S_{CHSH} & = X \otimes \left(Y + Y' \right) + X' \otimes (Y - Y') \\
& = \vec{a} \cdot \vec{\sigma} \otimes \left(\vec{b}\cdot \vec{\sigma} + \vec{b'} \cdot \vec{\sigma} \right) 
+ \vec{a'} \cdot \vec{\sigma} \otimes (\vec{b} \cdot \vec{\sigma} - \vec{b'} \cdot \vec{\sigma})
\end{align}
$$

where $\vec{a},\vec{b}, \vec{a'}, \vec{b'} \in \mathbb{R}^3$ and $|\vec{a}| = |\vec{b}| = |\vec{a'}| = |\vec{b'}| = 1$ .

In [2]:
def build_measure(phi: sym.Symbol, rho: sym.Symbol):
    """
    Construct a Pauli measurement given a unit 3-D vector `(phi, rho)`.
    The unit vector is (cos(phi) * cos(rho), cos(phi) * sin(rho), sin(phi)).
    """

    # Pauli matrices
    X = sym.Matrix([[0, 1], [1, 0]])
    Y = sym.Matrix([[0, -1j], [1j, 0]])
    Z = sym.Matrix([[1, 0], [0, -1]])

    # unit vector
    a = sym.cos(phi) * sym.cos(rho)
    b = sym.cos(phi) * sym.sin(rho)
    c = sym.sin(phi)

    return a * X + b * Y + c * Z

In [20]:
def get_CHSH():
    # Alice
    phi_1, rho_1 = sym.symbols(("phi^1_1:3", "rho^1_1:3"), real=True)
    # Bob
    phi_2, rho_2 = sym.symbols(("phi^2_1:3", "rho^2_1:3"), real=True)

    X_1 = build_measure(phi_1[0], rho_1[0])
    Y_1 = build_measure(phi_1[1], rho_1[1])
    X_2 = build_measure(phi_2[0], rho_2[0])
    Y_2 = build_measure(phi_2[1], rho_2[1])

    S_chsh = TensorProduct(X_1, Y_1 + Y_2) + TensorProduct(X_2, Y_1 - Y_2)
    return S_chsh

In [21]:
S_chsh = get_CHSH()

$$S(\rho) \equiv \max_{S} tr(S\rho)$$

In [18]:
def get_expectation(Rho: sym.Matrix, S: sym.Matrix):
    """
    Calculate $tr(S * Rho)$
    """
    return -(S * Rho).trace()

In [44]:
def get_gradient(e):
    return {p: sym.simplify(e.diff(p)) for p in e.free_symbols}

In [53]:
def optimizer_GCD(Rho, S, step=0.01, MAXEPOCH=100):
    e = get_expectation(Rho, S)
    e = sym.simplify(e)
    g = get_gradient(e)

    num_params = len(g)
    p_name = list(e.free_symbols)

    def cal(expr, x):
        return expr.subs({p: x[i] for (i, p) in enumerate(p_name)})
    
    def cal_dx(x):
        return [cal(g[p], x) for p in p_name]
    
    # init random
    x = np.random.uniform(0, 2 * np.pi, size=(num_params,))
    dx = np.array(cal_dx(x))
    e1 = cal(e, x)
    y = x - step * dx;
    e2 = cal(e, y)

    epoch = 0
    while np.abs(e1 - e2) > 1e-7 and epoch < MAXEPOCH:
        print(f"epoch={epoch:3}, e = {e1}")
        x = y
        e1 = e2
        dx = np.array(cal_dx(x))
        y = x - step * dx;
        e2 = cal(e, y)
        epoch += 1


$$|\psi\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}}$$

$$
\rho = |\psi\rangle \langle \psi| 
= \frac{1}{2} \begin{bmatrix} 0 \\ -1 \\ 1 \\ 0\end{bmatrix} 
\begin{bmatrix} 0 & -1 & 1 & 0  \end{bmatrix}
= \frac{1}{2} \begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 1 & -1 & 0 \\
0 & -1 & 1 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$$

In [35]:
Rho = 1 / 2 * sym.Matrix([[0, 0, 0, 0], [0, 1, -1, 0], [0, -1, 1, 0], [0, 0, 0, 0]])
Rho

Matrix([
[0,    0,    0, 0],
[0,  0.5, -0.5, 0],
[0, -0.5,  0.5, 0],
[0,    0,    0, 0]])

In [54]:
optimizer_GCD(Rho, S_chsh, MAXEPOCH=100000)

epoch=  0, e = -1.11633883256530
epoch=  1, e = -1.15800437013751
epoch=  2, e = -1.19873142145120
epoch=  3, e = -1.23851552913889
epoch=  4, e = -1.27735468389488
epoch=  5, e = -1.31524921272946
epoch=  6, e = -1.35220165897057
epoch=  7, e = -1.38821665552444
epoch=  8, e = -1.42330079288292
epoch=  9, e = -1.45746248332160
epoch= 10, e = -1.49071182267186
epoch= 11, e = -1.52306045097417
epoch= 12, e = -1.55452141323163
epoch= 13, e = -1.58510902138495
epoch= 14, e = -1.61483871852526
epoch= 15, e = -1.64372694625184
epoch= 16, e = -1.67179101597009
epoch= 17, e = -1.69904898481368
epoch= 18, e = -1.72551953676496
epoch= 19, e = -1.75122186944149
epoch= 20, e = -1.77617558691496
epoch= 21, e = -1.80040059883345
epoch= 22, e = -1.82391702602934
epoch= 23, e = -1.84674511271348
epoch= 24, e = -1.86890514528348
epoch= 25, e = -1.89041737770772
epoch= 26, e = -1.91130196338973
epoch= 27, e = -1.93157889336756
epoch= 28, e = -1.95126794066091
epoch= 29, e = -1.97038861054373
epoch= 30,

In [50]:
(np.sqrt(2) * 2)

2.8284271247461903